In [ ]:
from requests import get
from bs4 import BeautifulSoup
import io

In [ ]:
def parse_lyrics(song_link):
  response = get(song_link)
  html_soup = BeautifulSoup(response.text, 'html.parser')
  lyrics = html_soup.find_all('p')
  song = []
  for paragraph in lyrics:
    paragraph = str(paragraph).replace("<br/>", "\s").replace("</p>", "").replace("<p>", "").replace("!", "")
    song.append(paragraph)
  del song[-2:]
  song = '\n'.join(song)
  return song

In [ ]:
print("Connect with URL...")
url = 'https://www.nickcave.com/lyrics/'

In [ ]:
response = get(url)
url = 'https://www.nickcave.com/lyrics/'
html_soup = BeautifulSoup(response.text, 'html.parser')
albums = html_soup.find_all('div', class_ = 'lyric-album-list')
print(f'Number of available albums: {len(albums)}')


file  = io.open("songs.txt", "w", encoding="utf-8") 

print('Parse text...')

no_songs = 0

for album in albums:
  songs = album.find_all('a')
  for song in songs:
    song_link = song.get('href')
    file.write(parse_lyrics(song_link)+'\s')
    no_songs += 1

print(f"No. of downloaded songs: {no_songs}")

file.close()

print(f"'Songs.txt' file succesfully created!")

In [1]:
## Build tokenizer
from aitextgen import aitextgen
from aitextgen.tokenizers import train_tokenizer

file_name = "songs.txt"

train_tokenizer(file_name)
vocab_file = "aitextgen-vocab.json"
merges_file = "aitextgen-merges.txt"

INFO:aitextgen.tokenizers:Saving aitextgen-vocab.json and aitextgen-merges.txt to the current directory. You will need both files to build the GPT2Tokenizer.


In [2]:
## Train model
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

ai = aitextgen(vocab_file=vocab_file, merges_file=merges_file, config=GPT2ConfigCPU())

INFO:aitextgen:Constructing GPT-2 model from provided config.
INFO:aitextgen:Using a custom tokenizer.


In [3]:
## Create TokenDatasets, that builds datasets for training, processing them with the appropriate size.
from aitextgen.TokenDataset import TokenDataset

data = TokenDataset(file_name, vocab_file=vocab_file, merges_file=merges_file, block_size=64)

INFO:aitextgen.TokenDataset:Encoding 1,450 sets of tokens from songs.txt.


In [4]:
## Train!
ai.train(data, batch_size=16, num_steps=10000)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
No environment variable for node rank defined. Set as 0.
/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
\s And my girl\s O\s And it’s And I’t me\s I’s a long a morning\s And the night I’t you’m a my my love\s And I’t your road\s I’t got a head\s I’
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
\s A man in the pain, my little world\s The bathtub\s There baby’s But I don’t no long time, I ain’t got a little long
The night, my girl and a little eyes\s When she was a little heart of the man
I’t
3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
 up me\s You are all the sun, my baby and my hand\s And I’ll be my baby, my baby?\s We had a little Lord\s I comes\s This is a little little girl she will be my baby?\s There is a hands\s I love baby?\
4,000 steps reached: saving model to /trained_model
4,000 steps reached: generating sample texts.
 at\s She’ll get the sea\s I was all away
I’m a tiny little 

INFO:aitextgen:Saving trained model pytorch_model.bin to /trained_model


es\s I think I saw a-m the hammer
Am gonna the sea”\s And I felt a-bong?
And I’m gonna be a real in your hands\s Her hair was a children
I’m about this child\s I’ve been a little train\


In [5]:

ai.generate_to_file(n=3, prompt="We drowned", max_length=100, temperature=1.2)

INFO:aitextgen:Generating 3 texts to ATG_20200604_122536_59718989.txt
